In [1]:
import sys
import time
import matplotlib.pyplot as plt
import random
import numpy as np
import pickle
import gzip

# Práctica 1

## Parte 1

In [2]:
l = [[0, 10, 1, np.inf],
[np.inf, 0, 1, np.inf],
[np.inf, np.inf, 0, 1 ],
[np.inf, 1, np.inf, 0]]
m_g = np.array(l)

In [3]:
d_g = {0: {1: 10, 2:1}, 1: {2: 1},
2: {3: 1}, 3: {1: 1}}

In [4]:
m_g,d_g

(array([[ 0., 10.,  1., inf],
        [inf,  0.,  1., inf],
        [inf, inf,  0.,  1.],
        [inf,  1., inf,  0.]]),
 {0: {1: 10, 2: 1}, 1: {2: 1}, 2: {3: 1}, 3: {1: 1}})

In [5]:
'''
n_nodes: numero de nodos
sparse_factor: proporcion de ramas
max_weight: peso maximo
'''
def rand_matr_pos_graph(n_nodes, sparse_factor, max_weight=50., decimals=0):
    l = np.full((n_nodes,n_nodes),np.inf)
    branch_number = int(sparse_factor*(n_nodes)*(n_nodes-1))
    for count in range(0,branch_number):
        condition = True
        while(condition):
            i = random.randint(0,n_nodes-1)
            j = random.randint(0,n_nodes-1)
            if(j!=i and l[i,j] == np.inf):
                condition = False         
        weight = random.randint(1,max_weight)
        l[i,j] = weight
    #np.fill_diagonal(l,0)
    return l               
    

In [6]:
def cuenta_ramas(m_g):
    n_nodes = m_g.shape[0]
    branches = 0
    for i in range(0,n_nodes):
        for j in range(0,n_nodes):
            if ((m_g[i,j] != np.inf) and i != j):
                branches+=1
    return branches

print()

In [7]:
def m_g_sparse_factor(m_g):
    n_nodes = m_g.shape[0]
    return cuenta_ramas(m_g)/(n_nodes*(n_nodes-1))

def check_sparse_factor(n_grafos,n_nodes,sparse_factor):
    sparses = []
    for i in range(0,n_grafos):
        m = rand_matr_pos_graph(n_nodes,sparse_factor)
        sparses.append(m_g_sparse_factor(m))
    return np.mean(sparses)    

In [94]:
def m_g_2_d_g(m_g):
    n_nodes = m_g.shape[0]
    d_g = {}
    for i in range(0,n_nodes):
        d_g[i] = {}
        for j in range(0,n_nodes):
            if(m_g[i,j] != np.inf):
                d_g[i][j] = m_g[i,j]
                #d_g[i].update({j:m_g[i][j]})
    return d_g

In [95]:
def d_g_2_m_g(d_g):
    n_nodes = len(d_g.keys())
    m_g = np.full((n_nodes,n_nodes),np.inf)
    for node in d_g:
        for branch in d_g[node]:
            m_g[node,branch] = d_g[node][branch]
    return m_g

In [96]:
def save_object(obj, f_name="obj.pklz", save_path="."):
    f = gzip.open(save_path+f_name,"wb")
    pickle.dump(obj,f)
    f.close()

def read_object(f_name, save_path="."):
    f = gzip.open(save_path+f_name,"rb")
    obj = pickle.load(f)
    f.close()
    return obj

In [97]:
def d_g_2_TGF(d_g,f_name):
    f = open(f_name,"w")
    for node in d_g.keys():
        f.write("%d\n" % node)
    f.write("#\n")
    for node in d_g.keys():
        for branch in d_g[node]:
            #f.write("%d %d %l\n" % (node, branch, d_g[node][branch]))
            f.write("{0} {1} {2}\n".format(node, branch, d_g[node][branch]))
    f.close()

In [98]:
def TGF_2_d_g(f_name):
    f = open(f_name,"r")
    line = f.read().split('\n')
    d_g = {}
    aux = []
    for c in line:
        aux.append(c)
        if(c == "#"):
            break
        #d_g[int(c)] = {}
        n = int(c)
        d_g.update({n:{}})
        
    #splitted = line.strip("\n").split(" ")
    line = line[len(aux):-1]
    for c in line:
        splitted = c.split(" ")
        #d_g[int(splitted[0])][int(splitted[1])] = float(splitted[2])
        p1 = int(splitted[1])
        p2 = float(splitted[2])
        d_g[int(splitted[0])].update({p1:p2})
    f.close()
    return d_g
           


n_nodes = 5
sparse = 0.5
m = rand_matr_pos_graph(n_nodes,sparse)
print(m)
dicc = m_g_2_d_g(m)
print("[m_g_2_d_g(m)]:\n", dicc)
dicc[0][2] = 0.5
d_g_2_TGF(dicc,"pepitogrillo.txt")
print("[RES]:\n", TGF_2_d_g("pepitogrillo.txt"))

[[inf 10. 42. 42. inf]
 [inf inf inf inf 16.]
 [inf inf inf inf 33.]
 [11. 32. inf inf inf]
 [inf 47.  5. 44. inf]]
[m_g_2_d_g(m)]:
 {0: {1: 10.0, 2: 42.0, 3: 42.0}, 1: {4: 16.0}, 2: {4: 33.0}, 3: {0: 11.0, 1: 32.0}, 4: {1: 47.0, 2: 5.0, 3: 44.0}}
[RES]:
 {0: {1: 10.0, 2: 0.5, 3: 42.0}, 1: {4: 16.0}, 2: {4: 33.0}, 3: {0: 11.0, 1: 32.0}, 4: {1: 47.0, 2: 5.0, 3: 44.0}}


In [ ]:
def dijkstra(d_g, u):
    d_dist = {}
    d_prev = {}
    return d_dist, d_prev